In [1]:
import pandas as pd
import numpy as np

In [2]:
rate_df = pd.read_csv('ratings.csv')
movies_all_info = pd.read_csv('movie_info.csv',index_col = 'movieId')

In [3]:
users = rate_df['userId'].values
movies = rate_df['movieId'].values
ratings = rate_df['rating'].values

mid_index = movies_all_info.index
mt_table = ['(no genres listed)', 'Action','Adventure','Animation','Children','Comedy','Crime','Documentary',
           'Drama','Fantasy', 'Film-Noir','Horror','IMAX','Musical','Mystery','Romance','Sci-Fi','Thriller',
           'War','Western']


movies_all_info_table = np.array([[0.0]*22]*150000)            # [isMinied,Avg,mt*20]
user_trend_att_table = np.array([[0]*20]*(max(users)+1))    # [mt_att*20] 
user_trend_tol_table = np.array([[0.0]*20]*(max(users)+1))    # [mt_tol*20] 

In [4]:
cnt = 0
for uid,mid,rating in zip(users,movies,ratings):
    
    if movies_all_info_table[mid][0] == 1:
        rating_avg = movies_all_info_table[mid][1]
        rating_diff = rating - rating_avg
        
        mts = movies_all_info_table[mid][2:]==1
        user_trend_att_table[uid][mts] += 1
        user_trend_tol_table[uid][mts] += rating_diff
        
    elif mid in mid_index:
        movies_all_info_table[mid][0] = 1
        movies_all_info_table[mid][1] = movies_all_info.loc[mid]['avg']
        movies_all_info_table[mid][2:] = movies_all_info.loc[mid][mt_table]
        
        rating_avg = movies_all_info_table[mid][1]
        rating_diff = rating - rating_avg
        
        mts = movies_all_info_table[mid][2:]==1
        user_trend_att_table[uid][mts] += 1
        user_trend_tol_table[uid][mts] += rating_diff
        
    else:
        continue
        
    cnt+=1

In [25]:
uid_test = 1972

user_trend_tol_table[uid_test]/user_trend_att_table[uid_test],user_trend_att_table[uid_test]

/home/mvnl/anaconda3/envs/kkbox/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


(array([        nan,  0.32283686,  0.13326492, -0.69735405, -0.42782104,
         0.36720327,  0.42291583,  0.21205713, -0.10211184,  0.07317568,
        -0.05251987,  0.81973225, -0.8573937 , -0.78378132,  0.36730244,
        -0.14592217,  0.50549762,  0.46306383, -0.50240414,  0.19406781]),
 array([  0, 250, 189,  43,  86, 469, 178,  17, 495, 104,  18, 165,   4,
         47,  90, 172, 161, 305,  47,  20]))

In [39]:
user_trend_df_cnt = pd.DataFrame(data=user_trend_att_table,columns=[mt+'_cnt' for mt in mt_table])
user_trend_df_rat = pd.DataFrame(data=user_trend_tol_table,columns=[mt+'_rat' for mt in mt_table])

user_trend_df = pd.concat([user_trend_df_cnt,user_trend_df_rat],axis=1)

In [45]:
user_trend_df.to_csv('user_trend.csv',index=False)